# 고급 Pandas; 12.1 Categorical Data

In [1]:
import numpy as np
import pandas as pd
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
PREVIOUS_MAX_ROWS = pd.options.display.max_rows
pd.options.display.max_columns = 20
pd.options.display.max_rows = 20
pd.options.display.max_colwidth = 80
np.set_printoptions(precision=4, suppress=True)

In [2]:
# 이 절에서는 pandas의 Categorical형을 활용하여 pandas 메모리 사용량을 줄이고 성능을 개선할 수 있는 방법을 소개한다.
# 통계와 머신러닝에서 범주형 데이터를 활용하기 위한 도구들도 함께 소개하겠다.

- 12.1.1 개발 배경과 동기

In [3]:
# 하나의 컬럼 내에 특정 값이 반복되어 존재하는 경우는 흔하다. 
# 우리는 이미 배열 내에서 유일한 값을 추출하거나 특정 값이 얼마나 많이 존재하는지 확인할 수 있는 unique와 value_counts 같은 메서드를 공부했다.

In [4]:
import numpy as np; import pandas as pd

In [5]:
values = pd.Series(["apple", "orange", "apple",
                    "apple"] * 2) 

In [6]:
values

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
dtype: object

In [7]:
pd.unique(values)

array(['apple', 'orange'], dtype=object)

In [9]:
pd.value_counts(values)

apple     6
orange    2
dtype: int64

In [10]:
# 데이터웨어하우스, 분석 컴퓨팅 외 여러 다양한 데이터 시스템은 중복되는 데이터를 얼마나 효율적으로 저장하고 계산할 수 있는가를 중점으로 개발되었다.
# 데이터웨어하우스의 경우 구별되는 값을 담고 있는 차원 테이블과 그 테이블을 참조하는 정수키를 사용하는 것이 일반적이다.

In [14]:
values = pd.Series([0, 1, 0, 0] * 2)

In [15]:
dim = pd.Series(["apple", "orange"])

In [16]:
values

0    0
1    1
2    0
3    0
4    0
5    1
6    0
7    0
dtype: int64

In [17]:
dim

0     apple
1    orange
dtype: object

In [18]:
# take 메서드를 사용하면 Series 내에 저장된 원래 문자열을 구할 수 있다.

In [19]:
dim.take(values)

0     apple
1    orange
0     apple
0     apple
0     apple
1    orange
0     apple
0     apple
dtype: object

In [20]:
# 여기서 정수로 표현된 값은 범주형 또는 사전형 표기법이라고 한다. 
# 별개의 값을 담고 있는 배열은 범주, 사전 또는 단계 데이터라고 부른다. 
# 이 책에서는 이런 종류의 데이터를 Categorical 또는 범주형 데이터라고 부른다. 
# 범주형 데이터를 가리키는 정숫값은 범주 코드 또는 그냥 단순히 코드라고 한다. 

In [ ]:
# 범주형 표기법을 사용하면 분석 작업에 있어서 엄청난 성능 향상을 얻을 수 있다. 
# 범주 코드를 변경하지 않은 채로 범주형 데이터를 변형하는 것도 가능하다. 
# 비교적 적은 연산으로 수행할 수 있는 변형의 예는 다음과 같다. 

- 범주형 데이터의 이름 변경하기
- 기존 범주형 데이터의 순서를 바꾸지 않고 새로운 범주 추가하기

- 12.1.2 pandas의 Categorical

In [21]:
# pandas에는 정수 기반의 범주형 데이터를 표현(또는 인코딩)할 수 있는 Categorical형이라고 하는 특수한 데이터형이 존재한다.
# 앞서 살펴본 Series를 다시 보자.

In [22]:
fruits = ["apple", "orange", "apple", "apple"] * 2

In [23]:
N = len(fruits)

In [24]:
df = pd.DataFrame({"fruit": fruits,
                   "basket_id": np.arange(N),
                   "count": np.random.randint(3, 15, size=N),
                   "weight": np.random.uniform(0, 4, size=N)},
                  columns=["basket_id", "fruit", "count", "weight"])

In [25]:
df

,basket_id,fruit,count,weight
0,0,apple,5,3.858058
1,1,orange,8,2.612708
2,2,apple,4,2.995627
3,3,apple,7,2.614279
4,4,apple,12,2.990859
5,5,orange,8,3.845227
6,6,apple,5,0.033553
7,7,apple,4,0.425778


In [26]:
# 아래 예제에서 df["fruit"]는 파이썬 문자열 객체의 배열로, 아래 방법으로 쉽게 범주형 데이터로 변경할 수 있다.

In [27]:
fruit_cat = df["fruit"].astype("category")

In [28]:
fruit_cat

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [29]:
# fruit_cat의 값은 NumPy의 배열이 아니라 pandas.Categorical의 인스턴스다.

In [30]:
c = fruit_cat.values

In [31]:
type(c)

pandas.core.arrays.categorical.Categorical

In [32]:
# Categorical 객체는 categories와 codes 속성을 가진다.

In [33]:
c.categories

Index(['apple', 'orange'], dtype='object')

In [34]:
c.codes

array([0, 1, 0, 0, 0, 1, 0, 0], dtype=int8)

In [35]:
# 변경완료된 값을 대입함으로써 DataFrame의 컬럼을 범주형으로 변경할 수 있다.

In [36]:
df["fruit"] = df["fruit"].astype("category")

In [37]:
 df.fruit

0     apple
1    orange
2     apple
3     apple
4     apple
5    orange
6     apple
7     apple
Name: fruit, dtype: category
Categories (2, object): ['apple', 'orange']

In [38]:
# 파이썬 열거형에서 pandas.Categorical형을 직접 생성하는 것도 가능하다.

In [39]:
my_categories = pd.Categorical(["foo", "bar", "baz", "foo", "bar"])

In [40]:
my_categories

['foo', 'bar', 'baz', 'foo', 'bar']
Categories (3, object): ['bar', 'baz', 'foo']

In [41]:
# 기존에 정의된 범주와 범주 코드가 있다면 from_codes 함수를 이용해서 범주형 데이터를 생성하는 것도 가능하다.

In [42]:
categories = ["foo", "bar", "baz"]

In [43]:
codes = [0, 1, 2, 0, 0, 1]

In [44]:
my_cats_2 = pd.Categorical.from_codes(codes, categories)

In [45]:
my_cats_2

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo', 'bar', 'baz']

In [46]:
# 범주형으로 변경하는 경우 명시적으로 지정하지 않는 한 특정 순서를 보장하지 않는다.
# 따라서 categories 배열은 입력 데이터의 순서에 따라 다른 순서로 나타날 수 있다. 
# from_codes를 사용하거나 다른 범주형 데이터 생성자를 이용하는 경우, 순서를 지정할 수 있다.

In [47]:
ordered_cat = pd.Categorical.from_codes(codes, categories, 
                                        ordered=True)

In [48]:
ordered_cat

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [49]:
# 여기서 [foo < bar < baz]는 foo, bar, baz 순서를 가진다는 의미다. 
# 순서가 없는 범주형 인스턴스는 as_ordered 메서드를 이용해 순서를 가지도록 만들 수 있다.

In [50]:
my_cats_2.as_ordered()

['foo', 'bar', 'baz', 'foo', 'foo', 'bar']
Categories (3, object): ['foo' < 'bar' < 'baz']

In [51]:
# 여기서는 문자열만 예로 들었지만 범주형 데이터는 꼭 문자열일 필요는 없다. 
# 범주형 배열은 변경이 불가능한 값이라면 어떤 자료형이라도 포함할 수 있다.

- 12.1.3 Categorical 연산

In [52]:
# pandas에서 Categorical은 문자열 배열처럼 인코딩되지 않은 자료형을 사용하는 방식과 거의 유사하게 사용할 수 있다. 
# groupby 같은 일부 pandas 함수는 범주형 데이터에 사용할 때 더 나은 성능을 보여준다.
# ordered 플래그를 활용하는 함수들도 마찬가지다. 

In [53]:
# 임의의 숫자 데이터를 pandas.qcut 함수로 구분해보자. 
# 그렇게 하면 pandas.Categorical 객체를 반환한다.
# 책 초반부에서 pandas.cut 함수를 살펴봤지만 어떻게 범주형 데이터를 다루는지는 제대로 설명하지 않았다.

In [54]:
np.random.seed(12345)

In [55]:
draws = np.random.randn(1000)

In [56]:
draws[:5]

array([-0.2047,  0.4789, -0.5194, -0.5557,  1.9658])

In [57]:
# 이 데이터를 사분위로 나누고 통계를 내보자.

In [58]:
bins = pd.qcut(draws, 4)

In [59]:
bins

[(-0.684, -0.0101], (-0.0101, 0.63], (-0.684, -0.0101], (-0.684, -0.0101], (0.63, 3.928], ..., (-0.0101, 0.63], (-0.684, -0.0101], (-2.9499999999999997, -0.684], (-0.0101, 0.63], (0.63, 3.928]]
Length: 1000
Categories (4, interval[float64, right]): [(-2.9499999999999997, -0.684] < (-0.684, -0.0101] < (-0.0101, 0.63] < (0.63, 3.928]]

In [60]:
# 사분위 이름을 실제 데이터로 지정하는 것은 별로 유용하지 않은듯 보인다.
# qcut 함수의 labels 인자로 직접 이름을 지정하자.

In [61]:
bins = pd.qcut(draws, 4, labels=["Q1", "Q2", "Q3", "Q4"])

In [62]:
bins

['Q2', 'Q3', 'Q2', 'Q2', 'Q4', ..., 'Q3', 'Q2', 'Q1', 'Q3', 'Q4']
Length: 1000
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

In [63]:
bins.codes[:10]

array([1, 2, 1, 1, 3, 3, 2, 2, 3, 3], dtype=int8)

In [64]:
# bins에 이름을 붙이고 나면 데이터의 시작값과 끝값에 대한 정보를 포함하지 않으므로 groupby를 이용해서 요약 통계를 내보자.

In [65]:
bins = pd.Series(bins, name="quartile")

In [66]:
results = (pd.Series(draws)
           .groupby(bins)
           .agg(["count", "min", "max"])
           .reset_index())

In [67]:
results

,quartile,count,min,max
0,Q1,250,-2.949343,-0.685484
1,Q2,250,-0.683066,-0.010115
2,Q3,250,-0.010032,0.628894
3,Q4,250,0.634238,3.927528


In [68]:
# 결과에서 quartile 컬럼은 bins의 순서를 포함한 원래 범주 정보를 유지하고 있다.

In [69]:
results["quartile"]

0    Q1
1    Q2
2    Q3
3    Q4
Name: quartile, dtype: category
Categories (4, object): ['Q1' < 'Q2' < 'Q3' < 'Q4']

- categorical을 이용한 성능 개선

In [70]:
# 특정 데이터셋에 대해 다양한 분석을 하는 경우 범주형으로 변환하는 것만으로도 전체 성능을 개선할 수 있다.
# 범주형으로 변환한 DataFrame의 컬럼은 메모리도 훨씬 적게 사용한다.
# 소수의 독립적인 카테고리로 분류되는 천만 개의 값을 포함하는 Series를 살펴보자.

In [71]:
N = 10000000

In [72]:
draws = pd.Series(np.random.randn(N))

In [73]:
labels = pd.Series(["foo", "bar", "baz", "qux"] * (N // 4))

In [74]:
# labels를 categorical로 변환하자. 

In [75]:
categories = labels.astype("category")

In [76]:
# categories가 labels에 비해 훨씬 더 적은 메모리를 사용하는 것을 확인할 수 있다.

In [77]:
labels.memory_usage()

80000128

In [78]:
categories.memory_usage()

10000332

In [79]:
# 범주형으로 변환하는 과정이 그냥 이루어지는 것은 아니지만 이는 한 번만 변환하면 되는 일회성 비용이다.

In [80]:
%time _ = labels.astype("category")

CPU times: total: 375 ms
Wall time: 368 ms


In [81]:
# 범주형에 대한 그룹 연산은 문자열 배열을 사용하는 대신 정수 기반의 코드 배열을 사용하는 알고리즘으로 동작하므로 훨씬 빠르게 동작한다. 

- 12.1.4 Categorical 메서드

In [82]:
# 범주형 데이터를 담고 있는 Series는 특화된 문자열 메서드인 Series.str과 유사한 몇 가지 특수 메서드를 제공한다. 
# 이를 통해 categories와 codes에 쉽게 접근할 수 있다.
# 다음 Series를 살펴보자.

In [83]:
s = pd.Series(["a", "b", "c", "d"] * 2)

In [84]:
cat_s = s.astype("category")

In [85]:
cat_s

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [86]:
# 특별한 속성인 cat을 통해 categorical 메서드에 접근할 수 있다. 

In [87]:
cat_s.cat.codes

0    0
1    1
2    2
3    3
4    0
5    1
6    2
7    3
dtype: int8

In [88]:
cat_s.cat.categories

Index(['a', 'b', 'c', 'd'], dtype='object')

In [89]:
# 이 데이터의 실제 카테고리가 데이터에서 관측되는 4종류를 넘는 것을 이미 알고 있다고 가정하자. 
# 이 경우 set_categories 메서드를 이용해서 변경하는 것이 가능하다. 

In [90]:
actual_categories = ["a", "b", "c", "d", "e"]

In [91]:
cat_s2 = cat_s.cat.set_categories(actual_categories)

In [92]:
cat_s2

0    a
1    b
2    c
3    d
4    a
5    b
6    c
7    d
dtype: category
Categories (5, object): ['a', 'b', 'c', 'd', 'e']

In [93]:
# 데이터는 변함이 없지만 위에서 변경한 것처럼 새로운 카테고리가 추가되었다.
# 예를 들어 value_counts를 호출해보면 변경된 카테고리를 반영하고 있다.

In [95]:
cat_s.value_counts()

a    2
b    2
c    2
d    2
dtype: int64

In [96]:
cat_s2.value_counts()

a    2
b    2
c    2
d    2
e    0
dtype: int64

In [97]:
# 큰 데이터셋을 다룰 경우 categorical을 이용하면 메모리를 아끼고 메모리를 아끼고 성능도 개선할 수 있다.
# 분석 과정에서 큰 DataFrame이나 Series를 한번에 걸러내고 나면 실제로 데이터에는 존재하지 않는 카테고리가 남아있을 수 있다.
# 이 경우 remove_unused_categories 메서드를 이용해서 관측되지 않는 카테고리를 제거할 수 있다.

In [98]:
cat_s3 = cat_s[cat_s.isin(["a", "b"])]

In [99]:
cat_s3

0    a
1    b
4    a
5    b
dtype: category
Categories (4, object): ['a', 'b', 'c', 'd']

In [100]:
cat_s3.cat.remove_unused_categories()

0    a
1    b
4    a
5    b
dtype: category
Categories (2, object): ['a', 'b']

In [101]:
# [표 12-1]에 categorical 메서드의 종류를 나열해두었다. 페이지 492-493

- 모델링을 위한 더미값 생성하기

In [102]:
# 통계나 머신러닝 도구를 사용하다보면, 범주형 데이터를 더미값으로 변환(one-hot encoding이라고도 함)해야 하는 경우가 생긴다.
# 이를 위해 각각의 구별되는 카테고리를 컬럼으로 가지는 DataFrame을 생성하는데, 각 컬럼에는 해당 카테고리 여부에 따라 0과 1의 값을 가지게 된다.

# 앞서 살펴본 예제를 다시 살펴보자.

In [103]:
cat_s = pd.Series(["a", "b", "c", "d"] * 2, dtype="category")

In [104]:
# 7장에서 다루었듯이 pandas.get_dummies 함수는 이런 1차원 범주형 데이터를 더미값을 가지는 DataFrame으로 변환한다.

In [105]:
pd.get_dummies(cat_s)

,a,b,c,d
0,1,0,0,0
1,0,1,0,0
2,0,0,1,0
3,0,0,0,1
4,1,0,0,0
5,0,1,0,0
6,0,0,1,0
7,0,0,0,1
